In [1]:
import pandas as pd
import os
import re
from functools import reduce
import numpy as np

In [3]:
%matplotlib inline
def missing_report(df):    
    missing_df = df.isnull().sum(axis=0).reset_index()
    missing_df.columns = ['column_name', 'missing_count']
    missing_df['missing_ratio'] = missing_df['missing_count'] / df.shape[0]
    missing_df=missing_df[missing_df['missing_ratio']>0]
    missing_df=missing_df.sort_values(by=['missing_count'])
    if missing_df.shape[0]>=20:
        print(missing_df.iloc[-20:,])
    elif missing_df.shape[0]>=10:
        print(missing_df.iloc[-10:])
    else:
        print(missing_df)
    if missing_df.shape[0]<5:
        plt.figure(figsize=(2*missing_df.shape[0],6))
    elif missing_df.shape[0]>5 and missing_df.shape[0]<15:
        plt.figure(figsize=(12,10))
    else:
        plt.figure(figsize=(15,10))
    result=missing_df
    sns.set(style="white", context="talk")
    sns.barplot( missing_df['missing_count'], missing_df['column_name'],palette="BuGn_d")
    return result
state_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
extra_data1=pd.read_csv('./extra_feature1.csv')
extra_data2=pd.read_csv('./extra_features.csv')
extra_data2['state']=extra_data2['state'].str.upper()
extra_data2['state']=extra_data2['state'].map(state_dict)
extra_data1=extra_data1.dropna(axis=1,thresh=int(extra_data1.shape[0]*0.3))
extra_data2=extra_data2.dropna(axis=1,thresh=int(extra_data2.shape[0]*0.3))

In [4]:
extra_data1.to_csv('extra_feature1.csv',index=False)
extra_data2.to_csv('extra_feature2.csv',index=False)

In [18]:
gdp=pd.read_csv('gdp_all_year.csv')
pce=pd.read_csv('pce_all_year.csv')
pi=pd.read_csv('pi_all_year.csv')
target=pd.read_csv('target.csv')

In [10]:
#gdp=gdp.loc[gdp.year<=2014,:]

In [24]:
gdp.columns

Index(['GeoName', 'year', 'Compensation of employees',
       'Gross domestic product (GDP) by state', 'Gross operating surplus',
       'Per capita real GDP by state',
       'Quantity indexes for real GDP by state', 'Real GDP by state',
       'Subsidies', 'Taxes on production and imports',
       'Taxes on production and imports less subsidies'],
      dtype='object')

In [48]:
gdp=gdp.replace('(NA)',np.nan)
gdp_clean_list=['Compensation of employees','Subsidies','Taxes on production and imports','Gross operating surplus']
gdp.loc[:,gdp_clean_list]=gdp.loc[:,gdp_clean_list].astype(float)
#drop real gdp

In [75]:
#pce.corr()
#pce correlation acceptable

In [30]:
dfs = [gdp, pce, pi]
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on=['GeoName','year']), dfs)

In [17]:
df_final.head(5)

,GeoName,year,Compensation of employees,Gross domestic product (GDP) by state,Gross operating surplus,Per capita real GDP by state,Quantity indexes for real GDP by state,Real GDP by state,Subsidies,Taxes on production and imports,...,Income taxes (net of refunds),Other taxes,Other taxes,Federal government,Less: Personal current taxes,Local governments,State and local personal property taxes,State governments,Equals: Disposable personal income,Personal current taxes
0,Alabama,1997,61240,104309.0,36814,31465.0,81.042,137437.0,-328,6582,...,8558198.0,40573.0,15957.0,8558198.0,10612417.0,136089.0,44757.0,1873373.0,83209782.0,10612417.0
1,Alaska,1997,12205,25763.0,11234,62577.0,75.935,38358.0,-62,2386,...,1797971.0,1705.0,15139.0,1797971.0,1844364.0,4999.0,2786.0,38608.0,15203610.0,1844364.0
2,Arizona,1997,69886,132795.0,54249,34485.0,67.195,163353.0,-383,9044,...,10868970.0,16187.0,21430.0,10868970.0,12903928.0,17978.0,163562.0,1853418.0,94533284.0,12903928.0
3,Arkansas,1997,32852,60248.0,23415,30470.0,79.178,79256.0,-298,4279,...,4292918.0,3048.0,22535.0,4292918.0,5709269.0,3048.0,30510.0,1382793.0,46699786.0,5709269.0
4,California,1997,574394,1082051.0,441963,41314.0,70.134,1342133.0,-5725,71419,...,93479407.0,319603.0,69778.0,93479407.0,120765010.0,319603.0,1362268.0,25603732.0,761138752.0,120765010.0


In [31]:
target=target.groupby(['state','year']).sum().reset_index()
df_final=pd.merge(df_final,target,left_on=['GeoName','year'],right_on=['state','year'])

In [33]:
df_final.to_csv('all_year.csv',index=False)

In [2]:
import os
file_list=os.listdir('C:/Users/ttodd/OneDrive/Desktop/capstone/industry/')
industry.columns=['Year', 'State', 'firm type', 'NAICS', 'Total','<20', '20-99', '100-499', '500+', '<500']
industry_fillna=industry.loc[:,['Year','State','firm type']]
industry_fillna=industry_fillna.fillna(method='ffill')
industry.loc[:,['Year','State','firm type']]=industry_fillna
industry.to_csv('industry_target.csv',index=False)

In [23]:
core = df_final.corr() # df is the pandas dataframe
c1 = core.abs().unstack()
c1=c1[c1<=0.9]
c1.sort_values(ascending = False)

Veterans' benefits                                                                        Supplemental Nutrition Assistance Program (SNAP)                                            8.999955e-01
Supplemental Nutrition Assistance Program (SNAP)                                          Veterans' benefits                                                                          8.999955e-01
Real GDP by state                                                                         Earned Income Tax Credit (EITC)                                                             8.999320e-01
Earned Income Tax Credit (EITC)                                                           Real GDP by state                                                                           8.999320e-01
pce Less: Receipts from sales of goods and services by nonprofit institutions             Per capita net earnings                                                                     8.997622e-01
Per capita net earnings  

In [4]:
test_df=pd.read_csv('./households_housing_infrastructure_internet_investment_jobs_legal_manufacturing_military_nondurable goods_private_railroad.csv')
drop_list=['New Private Housing Units Authorized by Building Permits: 1-Unit Structures _y',
           'Durable Manufacturing Wages and Salaries ','Nondurable Manufacturing Wages and Salaries ']
test_df=test_df.drop(drop_list,axis=1)
test_df.to_csv('house_manu.csv',index=False)
test1=pd.read_csv('./wholesale_utilities_natural resources_advanced degree_air travel_arts_benefits_business_capital_commuting time_consumer_corporate_credits_deposits_education_energy_equipment_financial.csv')
test1_col=['state','year','New Private Housing Units Authorized by Building Permits _x','Homeownership Rate ', 'Home Vacancy Rate ', 'Median Household Income ',
       'All-Transactions House Price Index ','Job Losers and Persons Who Completed Temporary Jobs, as a Percent of the Civilian Labor Force ']
test_df=test_df.loc[:,test1_col]
test_df.columns=['state','year','New Private Housing Units Authorized by Building Permits','Homeownership Rate ', 'Home Vacancy Rate ', 'Median Household Income ',
       'All-Transactions House Price Index ','Job Losers and Persons Who Completed Temporary Jobs, as a Percent of the Civilian Labor Force ']
test2_col=['year',
       'state', 'CredAbility Consumer Distress Index  (DISCONTINUED)',
       'State Government Tax Collections, Corporation Net Income Taxes in Wisconsin','High School Graduate or Higher ',
       'People 25 Years and Over Who Have Completed an Advanced Degree ',
       'State Unemployment Benefits ',
       'Transfers less State Unemployment Benefits  (DISCONTINUED)',
       'Total Equity Capital for Commercial Banks ',
       'State Government Tax Collections, Public Utilities - Selective Sales Taxes ',
 ]
test1=test1.loc[:,test2_col]
final=pd.merge(test1,test_df,on=['state','year'],how='outer')
drop_na_thred=int(final.shape[0]*0.5)
final=final.dropna(axis=1,thresh=drop_na_thred)

In [6]:
final.to_csv('extra_features.csv',index=False)